### Load base model

In [ ]:
from ptlpinns.models import model, training
from ptlpinns.odes import forcing, numerical, equations
import numpy as np
import torch

name = "undamped_k12"
base_path = f"/home/dda24/PTL-PINNs/ptlpinns/models/train/{name}/"
base_name = f"model_{name}.pth"
base_model, training_log = model.load_model(base_path, base_name)

12 True True True 1.0 16 [256, 256, 512]


### Initialize transfer learning model

In [5]:
def transfer_model(training_log, base_model):

    bias = training_log['bias']
    use_sine = training_log['use_sine']
    use_fourier = training_log['use_fourier']
    scale = training_log['scale']
    n_frequencies = training_log['n_frequencies']
    hidden_layers = training_log['hidden_layers']

    if use_fourier:
        transfer_model = model.Multihead_model_fourier(k=1, bias=bias, use_sine=use_sine,
                                                use_fourier=use_fourier, scale=scale,
                                                n_frequencies=n_frequencies, HIDDEN_LAYERS=hidden_layers)
    else:
        transfer_model = model.Multihead_model_fourier(k=1, bias=bias, HIDDEN_LAYERS=hidden_layers)

    
    # Extract all layers except the final layers
    backbone = {k: v for k, v in base_model.state_dict().items() if not k.startswith("final_layers")}
    transfer_model.load_state_dict(backbone, strict=False)

    # freeze all layers except the final layers
    for name, param in transfer_model.named_parameters():
        if not name.startswith("final_layers"):
            param.requires_grad = False
    
    return transfer_model

In [6]:
transfer_model = transfer_model(training_log, base_model)

### Transfer learning parameters

In [ ]:
w_transfer = [1]
zeta_transfer = [0]
q = 3
N = 512
t_span = (0, 15)
t_eval = np.linspace(t_span[0], t_span[1], N)
ic = [[1, 0]]

In [ ]:
ode = equations.ode_oscillator_1D(w_0=w_transfer, zeta=zeta_transfer, forcing_1D = lambda t: np.zeros_like(t), q=q, epsilon=0.5)
numerical_solution = numerical.solve_ode_equation(ode, (t_eval[0], t_eval[-1]), t_eval, ic)

### Training

In [ ]:
def head_parameters(model, head_prefix="final_layers"):
    return [p for n, p in model.named_parameters() if n.startswith(head_prefix) and p.requires_grad]


In [ ]:
def compute_transfer_learning(model, optimizer, num_iter, equation_functions, initial_condition_functions, forcing_functions,
           N=512, t_span=(0, 1), every=100, ode_weight=1, ic_weight=1, method='equally-spaced-noisy'):
    
       for it in range(1, num_iter):
              optimizer.zero_grad()
              total, ode, ic, _ = training.loss(
              model=model, N=N, t_span=t_span,
              equation_functions=equation_functions,
              initial_condition_functions=ic_functions,
              forcing_functions=forcing_functions,
              ode_weight=ode_weight, ic_weight=ic_weight, method=method
              )
              total.backward()
              optimizer.step()

       if it % every == 0:
            print(f"[Head] {it} | total {total.item():.3e} | ode {ode.item():.3e} | ic {ic.item():.3e}")

In [ ]:
import torch

@torch.no_grad()
def evaluate_error(model, t_eval, numerical_solution):

    model.eval()
    pred_stack, _ = model(t_eval)  # shape: (k, N, 2)
    pred = pred_stack[0]              # (N, 2) since k=1
    x_pred  = pred[:, 0].contiguous()
    v_pred = pred[:, 1].contiguous()

    y_true, yp_true = numerical_solution(t_eval)  # both shape [N]
    # RMSEs
    rmse_y  = torch.sqrt(torch.mean((x_pred  - y_true )**2))
    rmse_yp = torch.sqrt(torch.mean((v_pred - yp_true)**2))
    abs_rmse = 0.5*(rmse_y + rmse_yp)

    # relative RMSE normalized by signal RMS + tiny epsilon
    eps = 1e-12
    rel_y  = rmse_y  / (torch.sqrt(torch.mean(y_true**2))  + eps)
    rel_yp = rmse_yp / (torch.sqrt(torch.mean(yp_true**2)) + eps)
    rel_rmse = 0.5*(rel_y + rel_yp)

    # scalar floats
    return float(rel_rmse.item()), float(abs_rmse.item())
